# Networks

This section focuses on docker networks in docker.

In this section, it is crucial to obtain the name of the network to which the container is connected. To achieve this, you can use the following command:

`docker inspect -f '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' <container name>`

---

The following cell prints the network name for the container just created.

In [1]:
docker run --name network_test -itd alpine &> /dev/null
docker inspect \
    -f '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_test
docker stop network_test &> /dev/null

bridge 


Containers connect to the `brindge` network by default. As a result we got the message `bridge`.

## Connect container

To effectively connect Docker containers to networks, you need to understand the following options:

- The `--network <network>` parameter in the `docker run` command, which specifies the network a container should connect to.
- The `docker network connect <network> <container>` command, used for connecting an existing container to a specified network.
- The `docker network disconnect <network> <container>` command, used for disconnecting a container from a network.

---

As an example, let's look at the `test_network` created in the following cell.

In [22]:
docker network create test_network &> /dev/null

: 1

---

The following example shows how to connect the container to the `test_network` using the `--network` parameter of the `docker run command`. It connects a new container to the network and then prints it's network name corresponding to the container.

In [23]:
docker run -itd --rm \
    --network test_network \
    --name test_container \
    alpine &> /dev/null

docker inspect \
    -f '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    test_container

test_network 


---

Now, using `docker disconnect`, we can disconnect the container from the `test_network'. And again - print out the container's current network.

In [19]:
docker network disconnect test_network test_container
docker inspect \
    -f '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    test_container

So now we consider that containers simply have no network.

---

Here is the example of connecting the container back to the network using command `docker network connect`.

In [20]:
docker network connect test_network test_container
docker inspect \
    -f '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    test_container

test_network 


So we got `test_network` back in the output.

Remove all temporary entities to keep the environment clean.

In [21]:
docker stop test_container &> /dev/null
docker network rm test_network &> /dev/null

## Default networks

For more infomation check [specific page](networks/default_networks.ipynb).

Docker has three default networks: `bridge`, `host`, and `none`. The following command lists all of them:

In [1]:
docker network ls \
    --filter name=bridge \
    --filter name=host \
    --filter name=none

NETWORK ID     NAME      DRIVER    SCOPE
77a0faf549ac   bridge    bridge    local
80c3e6772c8c   host      host      local
45bb103d970f   none      null      local


Here's a brief description of each default Docker network:

1. **Bridge**: The default network for containers when none is specified. Containers on the same bridge network can communicate with each other using their container names or IP addresses. It provides basic isolation between containers on different bridge networks.

2. **Host**: Containers share the host’s network stack and IP address. This means the container will have direct access to the host’s network interfaces and ports, which can be useful for performance but offers less isolation.

3. **None**: Disables all networking for the container. Containers on the `none` network cannot communicate with other containers or the outside world, making it suitable for scenarios where networking is not needed.